#### Importing Libs

In [1]:
from core.PipelineProject import PrepareDataAndTrainingModels
from core.SQLeto import SQLeto
from decouple import config
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#### Load enviroment vars

In [2]:
RAW_DATA_PATH = config('RAW_DATA_PATH')
DATABASE_POSTGRES = config('DATABASE_POSTGRES')
USER_POSTGRES = config('USER_POSTGRES')
PWD_POSTGRES = config('PWD_POSTGRES')

#### Load data from POTSGRES

In [3]:
# Create object to access database
sqleto = SQLeto(
    database=DATABASE_POSTGRES, 
    user=USER_POSTGRES,
    password=PWD_POSTGRES)
    
# Getting data in a dataframe
data = sqleto.execute_DQL(
    '''
        SELECT * FROM creditcard
    '''
)

In [4]:
data.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


#### Load models

In [5]:
# Models to test
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

#### Load evaluation metrics

In [6]:
from sklearn.metrics import (f1_score,
                             roc_auc_score,
                             recall_score,
                             precision_score)


#### Load scaler

In [7]:
# Scaler
from sklearn.preprocessing import RobustScaler

#### Load balancer

In [8]:
# Balancer
from imblearn.over_sampling import SMOTE

#### Use Pipeline to avaliate Models

In [9]:
pipe_2 = PrepareDataAndTrainingModels(
  dataframe=data,
    target="Class",
    scaler=RobustScaler(),
    models=[
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3)),
    ],
    cv_score_metric=None,    
    score_metric = (f1_score,
                    roc_auc_score,
                    recall_score,
                    precision_score)
)

In [10]:
pipe_2.splitting_data(standart_scale=True)

In [11]:
param = {'n_estimators':[20,50,100,300], 'learning_rate':[0.5,1,1.5]}
pipe_2.fit_models(grid_search_cv=True,cv=5,param=param)

ValueError: If using all scalar values, you must pass an index